In [1]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import warnings
import sys
sys.path.insert(0, '../')
from reservoirpy.datasets import japanese_vowels
from reservoirpy import set_seed, verbosity
from reservoirpy.observables import nrmse, rsquare

from sklearn.metrics import accuracy_score

set_seed(42)
verbosity(0)

0

In [2]:
X_train, Y_train, X_test, Y_test = japanese_vowels()

In [4]:
from reservoirpy.nodes import Reservoir, Ridge, Input

In [6]:
source = Input()
reservoir = Reservoir(500, sr=0.9, lr=0.1)
scikit = False
if scikit:
    readout = ElasticNet(alpha=0.001)
else:
    readout = Ridge(ridge=1.0)    

model = source >> reservoir >> readout

In [7]:
class ReduceTimesteps:
    def reduce(self, states, reduce=None):
        if reduce == 'mean':
            states = np.mean(states, axis=0)
        elif reduce == 'median':
            states = np.median(states, axis=0)
        elif reduce == 'peak':
            magnitudes = np.sum(states, axis=1)
            peak_pos = np.argmax(magnitudes)
            states = states[peak_pos]
        else:
            raise ValueError(f"reduce cannot be {reduce}. It must be 'mean','median' or 'peak'")
        return states
    
    def forward(self, X_train, window=None, reduce=None):
        states_train = []
        for x in X_train:
            states = reservoir.run(x, reset=True)
            import pdb;pdb.set_trace()
            if window != 'all':
                states = states[-window:].squeeze()
            if window != 1:
                states = self.reduce(states, reduce=reduce)
            states_train.append(states[None, :])
        return states_train

In [8]:
window = 1
reduce = None
states_train = ReduceTimesteps().forward(X_train, window=window, reduce=reduce)
if scikit:
    readout.fit(np.array(states_train).squeeze(), np.array(Y_train).squeeze())
else:
    readout.fit(states_train, Y_train)

> <ipython-input-7-ac374228ce22>(20)forward()
     18             states = reservoir.run(x, reset=True)
     19             import pdb;pdb.set_trace()
---> 20             if window != 'all':
     21                 states = states[-window:].squeeze()
     22             if window != 1:

ipdb> states.shape
(20, 500)
ipdb> x.shape
(20, 12)
ipdb> c
> <ipython-input-7-ac374228ce22>(19)forward()
     17         for x in X_train:
     18             states = reservoir.run(x, reset=True)
---> 19             import pdb;pdb.set_trace()
     20             if window != 'all':
     21                 states = states[-window:].squeeze()

ipdb> q


BdbQuit: 

In [61]:
states_test = ReduceTimesteps().forward(X_test, window=window, reduce=reduce)
Y_pred = []
for state in states_test:
    y = readout.run(state[-1, np.newaxis])
    Y_pred.append(y)
len(Y_pred)

370

In [62]:
Y_pred_class = [np.argmax(y_p) for y_p in Y_pred]
Y_test_class = [np.argmax(y_t) for y_t in Y_test]

score = accuracy_score(Y_test_class, Y_pred_class)

print("Accuracy: ", f"{score * 100:.3f} %")

Accuracy:  98.108 %
